In [ ]:
import urllib.request
from urllib.error import  URLError
import re
import pickle

webitems = []
def craw_url(url, domain):
    global crawlerbacklog

    if(len(crawlerbacklog)>100):
        return
    if(url in crawlerbacklog and crawlerbacklog[url] == 1):
        return
    else:
        crawlerbacklog[url] = 1
        
    try:
        page = urllib.request.urlopen(url)
        code=page.getcode()
        if(code == 200):
            content=page.read()
            content_string = content.decode("utf-8")
            regexp_header = re.compile('<h1>(?P<text>(.*))</h1>')           
            regexp_url = re.compile("http://"+domain+"[/\w+]*")

            result = regexp_header.search(content_string, re.IGNORECASE)
            
            if result:
                text = result.group("text")
                webitems.append((url, text))

            for (urls) in re.findall(regexp_url, content_string):
                if(urls  not in crawlerbacklog or crawlerbacklog[urls] != 1):
                    crawlerbacklog[urls] = 0
                    craw_url(urls, domain)
    except URLError as e:
        print("error")

    f = open('web_data.pickle', 'bw')
    pickle.dump(webitems, f)
    f.close()    

crawlerbacklog = {}

seed = "http://www.newhaven.edu/"

crawlerbacklog[seed]=0

craw_url(seed, "www.newhaven.edu")
